## NHK News Web / Title+Summary+Detail dataset

Website: https://www3.nhk.or.jp/news

### Approach

This website doesn't have archive page of news. So, it's a bit diffucult to get old articles.

But the website has some api endpoints currently known:

- Category: https://www3.nhk.or.jp/news/archtag/00000{:02d}_00{:02d}.json
- News by region: https://www3.nhk.or.jp/news/archtag/00022{:02d}_00{:02d}.json

By getting these API responses to get the url to the news article.


## Setup


In [40]:
import json
import requests
from bs4 import BeautifulSoup
from datasets import load_dataset, Dataset
from tqdm import tqdm
import time

## Cache URLs


In [32]:
items = []

client = requests.session()

print("main tags")

for i in range(1, 11):
    for j in range(1, 50):
        url = f"https://www3.nhk.or.jp/news/archtag/00000{i:02d}_00{j:02d}.json"
        print(url)
        res = client.get(url)
        res.encoding = res.apparent_encoding
        if res.status_code != 200:
            break
        data = json.loads(res.text)
        for item in data["channel"]["item"]:
            items.append(item)

        print(len(items))

print("都道府県")

for i in range(1, 48):
    for j in range(1, 50):
        url = f"https://www3.nhk.or.jp/news/archtag/00000{i:02d}_00{j:02d}.json"
        print(url)
        res = client.get(url)
        res.encoding = res.apparent_encoding
        if res.status_code != 200:
            break
        data = json.loads(res.text)
        for item in data["channel"]["item"]:
            items.append(item)

        print(len(items))

print("Done!")
print(len(items))

main tags
https://www3.nhk.or.jp/news/archtag/0000001_0001.json
100
https://www3.nhk.or.jp/news/archtag/0000001_0002.json
200
https://www3.nhk.or.jp/news/archtag/0000001_0003.json
300
https://www3.nhk.or.jp/news/archtag/0000001_0004.json
400
https://www3.nhk.or.jp/news/archtag/0000001_0005.json
500
https://www3.nhk.or.jp/news/archtag/0000001_0006.json
600
https://www3.nhk.or.jp/news/archtag/0000001_0007.json
700
https://www3.nhk.or.jp/news/archtag/0000001_0008.json
800
https://www3.nhk.or.jp/news/archtag/0000001_0009.json
900
https://www3.nhk.or.jp/news/archtag/0000001_0010.json
1000
https://www3.nhk.or.jp/news/archtag/0000001_0011.json
1100
https://www3.nhk.or.jp/news/archtag/0000001_0012.json
1200
https://www3.nhk.or.jp/news/archtag/0000001_0013.json
1300
https://www3.nhk.or.jp/news/archtag/0000001_0014.json
1385
https://www3.nhk.or.jp/news/archtag/0000001_0015.json
https://www3.nhk.or.jp/news/archtag/0000002_0001.json
1485
https://www3.nhk.or.jp/news/archtag/0000002_0002.json
1585
h

## Get news body

This takes a few minutes...


In [35]:
BASE_URL = "https://www3.nhk.or.jp"

In [44]:
# item にそれぞれ html を追加していく
for item in tqdm(items):
    # すでに html がある場合はスキップ
    if "html" in item:
        continue

    url = BASE_URL + item["link"]
    # print(url)
    res = client.get(url)
    if res.status_code != 200:
        if res.status_code == 404:
            # print("404")
            continue
        print("error!!!")
        print(res.status_code)
        break
    soup = BeautifulSoup(res.content, "lxml", from_encoding="utf-8")
    item["html"] = soup.prettify()

    time.sleep(0.1)  # avoid 429

100%|██████████| 16397/16397 [09:49<00:00, 27.81it/s]  


In [45]:
items[0]

{'id': '427766',
 'title': 'ヤクルト新人入団発表 ドラフト1位西舘昂汰「新人王とりたい」',
 'pubDate': 'Mon, 04 Dec 2023 19:53:00 +0900',
 'cate': '7',
 'cate_group': ['7'],
 'link': '/news/html/20231204/k10014277661000.html',
 'imgPath': '/news/html/20231204/K10014277661_2312041731_1204173643_01_02.jpg',
 'iconPath': '/news/html/20231204/K10014277661_2312041731_1204173643_01_02.jpg',
 'videoPath': '',
 'videoDuration': '0',
 'relationNews': [],
 'html': '<!DOCTYPE HTML>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7 eq-ie6"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8 eq-ie7"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9 eq-ie8"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="no-js">\n <!--<![endif]-->\n <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# website: http://ogp.me/ns/website#">\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="requiresActiveX=true" http-equiv="X-UA-Compatible

## Parse HTML


In [47]:
# 本文を取得して追加
for item in tqdm(items):
    # print(item["link"])
    if "html" not in item:
        continue
    soup = BeautifulSoup(item["html"], "lxml")
    summary_el = soup.select_one("p.content--summary")
    if summary_el is not None:
        summary = summary_el.get_text(strip=True)
    else:
        summary = None

    detail = soup.select_one("div.content--detail-more")
    if detail is not None:
        detail = detail.get_text(strip=True)
    else:
        detail = None

    item["summary"] = summary
    item["detail"] = detail

100%|██████████| 16397/16397 [01:14<00:00, 220.26it/s]


In [48]:
items[250]

{'id': '421002',
 'title': '【プロ野球結果】楽天3位浮上 パ2位-4位までゲーム差1の大混戦',
 'pubDate': 'Thu, 28 Sep 2023 23:39:00 +0900',
 'cate': '7',
 'cate_group': ['7'],
 'link': '/news/html/20230928/k10014210021000.html',
 'imgPath': '/news/html/20230928/K10014210021_2309282336_0928233918_01_02.jpg',
 'iconPath': '/news/html/20230928/K10014210021_2309282336_0928233918_01_02.jpg',
 'videoPath': '',
 'videoDuration': '0',
 'relationNews': [],
 'html': '<!DOCTYPE HTML>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7 eq-ie6"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8 eq-ie7"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9 eq-ie8"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="no-js">\n <!--<![endif]-->\n <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# website: http://ogp.me/ns/website#">\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="requiresActiveX=true" http-equiv="X-UA-Compatib

### Filter failed items


In [49]:
# filter items which has no html
items = [item for item in items if "html" in item]

In [50]:
ds = Dataset.from_list(items)
ds

Dataset({
    features: ['id', 'title', 'pubDate', 'cate', 'cate_group', 'link', 'imgPath', 'iconPath', 'videoPath', 'videoDuration', 'relationNews', 'html', 'summary', 'detail'],
    num_rows: 16376
})

## Upload to huggingface


In [51]:
SAVE_REOO_ID = "nhk-title-sum-detail"

In [52]:
ds.push_to_hub(
    SAVE_REOO_ID,
    private=True,
    config_name="all",
)

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

In [54]:
simple_ds = ds.remove_columns(
    [
        "cate",
        "cate_group",
        "imgPath",
        "iconPath",
        "videoPath",
        "videoDuration",
        "relationNews",
        "html",
    ]
)
simple_ds = simple_ds.filter(lambda x: x["summary"] is not None)
simple_ds = simple_ds.filter(lambda x: x["detail"] is not None)
simple_ds

Filter:   0%|          | 0/16376 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14590 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'pubDate', 'link', 'summary', 'detail'],
    num_rows: 13706
})

In [59]:
simple_ds.push_to_hub(
    SAVE_REOO_ID,
    private=True,
    config_name="default",
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

In [85]:
simple_ds[12345]

{'id': '275197',
 'title': 'ユニセフ 各国に学校再開求める “感染拡大証拠ほとんどなし”',
 'pubDate': 'Tue, 08 Dec 2020 12:08:00 +0900',
 'link': '/news/html/20201208/k10012751971000.html',
 'summary': 'ユニセフ＝国連児童基金は8日、学校が新型コロナウイルスの感染の拡大を招いているという証拠はほとんどないとして、学校の再開を優先させるよう各国政府に求める声明を出しました。',
 'detail': '声明では、今月1日時点で児童や生徒の5人に1人、3億2000万人が休校措置の影響を受けていて、先月1日時点に比べておよそ9000万人増えたとしています。一方で、各国のデータを用いた最新の研究では、学校の状況と地域の感染率には何の関連性も示されておらず、学校が感染の拡大を招いているという証拠はほとんどないとしています。そのうえで、あまりにも多くの学校が不必要に休校になっている懸念があるとして、各国政府に学校の再開を優先させ、安全を確保するための措置を可能なかぎりとるよう求めています。また、学校の再開計画にはリモート学習を利用できるようにすることなども盛り込むよう求めています。ユニセフのジェンキンス本部教育グローバルチーフは「最後の手段ではなく、最初の手段として休校措置をとるという、憂慮すべき傾向が再び見られるようになってきている。子どもたちは学習、心身の健康と安全に深刻な影響を受け続けている」としています。'}